# SSD300 Training Tutorial

This tutorial explains how to train an SSD300 on the Pascal VOC datasets. The preset parameters reproduce the training of the original SSD300 "07+12" model. Training SSD512 works simiarly, so there's no extra tutorial for that. The same goes for training on other datasets.

You can find a summary of a full training here to get an impression of what it should look like:
[SSD300 "07+12" training summary](https://github.com/pierluigiferrari/ssd_keras/blob/master/training_summaries/ssd300_pascal_07%2B12_training_summary.md)

In [1]:
from keras.optimizers import Adam, SGD
from keras.callbacks import Callback, ModelCheckpoint, LearningRateScheduler, TerminateOnNaN, CSVLogger, EarlyStopping, TensorBoard
from keras import backend as K
from keras.models import load_model
from math import ceil
import numpy as np
from matplotlib import pyplot as plt
from keras.models import Model
from matplotlib import pyplot as plt
from keras.preprocessing import image
from imageio import imread

from models.keras_ssd300_mod import ssd_300
from keras_loss_function.keras_ssd_loss_mod import SSDLoss
from keras_loss_function.keras_ssd_loss_proj_reformed import SSDLoss_proj

from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_input_encoder_mod import SSDInputEncoder
from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_geometric_ops import Resize_Modified
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels_Modified
from data_generator.data_augmentation_chain_original_ssd import SSDDataAugmentation_modified
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.data_augmentation_chain_original_ssd import SSDDataAugmentation

from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms
from bounding_box_utils.bounding_box_utils import iou, convert_coordinates
from ssd_encoder_decoder.matching_utils import match_bipartite_greedy, match_multi
import random
import tensorflow as tf


Using TensorFlow backend.


## 0. Preliminary note

All places in the code where you need to make any changes are marked `TODO` and explained accordingly. All code cells that don't contain `TODO` markers just need to be executed.

## 1. Set the model configuration parameters


In [2]:
img_height = 300 # Height of the model input images
img_width = 600 # Width of the model input images
img_channels = 3 # Number of color channels of the model input images
mean_color = [123, 117, 104] # The per-channel mean of the images in the dataset. Do not change this value if you're using any of the pre-trained weights.
swap_channels = [2, 1, 0] # The color channel order in the original SSD is BGR, so we'll have the model reverse the color channel order of the input images.
n_classes = 1 # Number of positive classes, e.g. 20 for Pascal VOC, 80 for MS COCO
scales_pascal = [0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05] # The anchor box scaling factors used in the original SSD300 for the Pascal VOC datasets
scales_coco = [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05] # The anchor box scaling factors used in the original SSD300 for the MS COCO datasets
scales = scales_pascal
aspect_ratios = [[1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5]] # The anchor box aspect ratios used in the original SSD300; the order matters
two_boxes_for_ar1 = True            # print(y_encoded)

steps = [8, 16, 32, 64, 100, 300] # The space between two adjacent anchor box center points for each predictor layer.
offsets = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5] # The offsets of the first anchor box center points from the top and left borders of the image as a fraction of the step size for each predictor layer.
clip_boxes = False # Whether or not to clip the anchor boxes to lie entirely within the image boundaries
variances = [0.1, 0.1, 0.2, 0.2] # The variances by which the encoded target coordinates are divided as in the original implementation
normalize_coords = True


## 2. Build or load the model

You will want to execute either of the two code cells in the subsequent two sub-sections, not both.

In [3]:

K.clear_session() # Clear previous models from memory.

model = ssd_300(image_size=(img_height, img_width, img_channels),
                n_classes=n_classes,
                mode='training',
                l2_regularization=0.0005,
                scales=scales,
                aspect_ratios_per_layer=aspect_ratios,
                two_boxes_for_ar1=two_boxes_for_ar1,
                steps=steps,
                offsets=offsets,
                clip_boxes=clip_boxes,
                variances=variances,
                normalize_coords=normalize_coords,
                subtract_mean=mean_color,
                swap_channels=swap_channels)
weights_path = 'weights/VGG_ILSVRC_16_layers_fc_reduced.h5'

model.load_weights(weights_path, by_name=True)


/scratch/gits/ssd_keras_double/models/keras_ssd300_mod.py:578: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  model = Model(input=[x],output=[mbox_conf, mbox_loc, mbox_priorbox])


In [4]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 300, 600, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 300, 600, 3)  0                                            
__________________________________________________________________________________________________
identity_layer__1 (Lambda)      (None, 300, 600, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
identity_layer__2 (Lambda)      (None, 300, 600, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
input_mean

In [5]:
def Accuracy(y_true, y_pred):
    y_true = y_true[:,:,:18]
    y_pred = y_pred[:,:,:18]

    return K.mean(K.equal(K.argmax(y_true[:,:,:-4], axis=-1),
                  K.argmax(y_pred[:,:,:-4], axis=-1)))

adam = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

ssd_loss1 = SSDLoss(neg_pos_ratio=3, alpha=1.0)
ssd_loss2 = SSDLoss(neg_pos_ratio=3, alpha=1.0)
ssd_loss3 = SSDLoss_proj(neg_pos_ratio=3, alpha=1.0)
ssd_loss4 = SSDLoss_proj(neg_pos_ratio=3, alpha=1.0)

losses = {
    "predictions_1": ssd_loss1.compute_loss,
    "predictions_2": ssd_loss2.compute_loss,
    "predictions_1_to_2": ssd_loss2.compute_loss,
    "predictions_2_to_1": ssd_loss2.compute_loss

}
lossWeights = {"predictions_1": 1.0,"predictions_2": 1.0,"predictions_1_to_2": 1.0,"predictions_2_to_1": 1.0}

MetricstDict = {"predictions_1": Accuracy,"predictions_2": Accuracy}

model.compile(optimizer=adam, loss=losses, loss_weights=lossWeights, metrics=MetricstDict) 


In [6]:
train_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)
val_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)

val_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)
val_dataset_1 = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)


VOC_2007_images_dir      = '../datasets/Images/'
VOC_2007_annotations_dir      = '../datasets/VOC/Pasadena/Annotations_Multi/'

VOC_2007_trainval_image_set_filename = '../datasets/VOC/Pasadena/ImageSets/Main/reid_neu/train_few.txt'
VOC_2007_val_image_set_filename      = '../datasets/VOC/Pasadena/ImageSets/Main/reid_neu/val_few.txt'
VOC_2007_test_image_set_filename     = '../datasets/VOC/Pasadena/ImageSets/Main/reid_neu/test_few.txt'

# VOC_2007_trainval_image_set_filename = '../datasets/VOC/Pasadena/ImageSets/Main/reid_neu/train.txt'
# VOC_2007_val_image_set_filename      = '../datasets/VOC/Pasadena/ImageSets/Main/reid_neu/val.txt'
# VOC_2007_test_image_set_filename     = '../datasets/VOC/Pasadena/ImageSets/Main/reid_neu/test.txt'


classes = ['background',
           'tree']

train_dataset.parse_xml(images_dirs=[VOC_2007_images_dir],
                        image_set_filenames=[VOC_2007_trainval_image_set_filename],
                        annotations_dirs=[VOC_2007_annotations_dir],
                        classes=classes,
                        include_classes='all',
                        exclude_truncated=False,
                        exclude_difficult=False,
                        ret=False)


val_dataset.parse_xml(images_dirs=[VOC_2007_images_dir],
                      image_set_filenames=[VOC_2007_val_image_set_filename],
                      annotations_dirs=[VOC_2007_annotations_dir],
                      classes=classes,
                      include_classes='all',
                      exclude_truncated=False,
                      exclude_difficult=True,
                      ret=False)

Processing image set 'val_few.txt': 100%|██████████| 2/2 [00:00<00:00, 48.73it/s]


### 2.2 Load a previously created model


In [7]:
# Define a learning rate schedule.

def lr_schedule(epoch):
    if epoch < 80:
        return 0.001
    elif epoch < 100:
        return 0.0001
    else:
        return 0.00001

In [8]:

neg_pos_ratio = 3
n_neg_min = 0
alpha = 1

def smooth_L1_loss(y_true, y_pred):
    absolute_loss = tf.abs(y_true - y_pred)
    square_loss = 0.5 * (y_true - y_pred)**2
    l1_loss = tf.where(tf.less(absolute_loss, 1.0), square_loss, absolute_loss - 0.5)
    return tf.reduce_sum(l1_loss, axis=-1)

def log_loss(y_true, y_pred):

    y_pred = tf.maximum(y_pred, 1e-15)
    # Compute the log loss
    log_loss = -tf.reduce_sum(y_true * tf.log(y_pred), axis=-1)
    return log_loss


def compute_loss(y_true, y_pred):
    def gt_rem(pred, gt):
        print("gt_rem")
        print("pred: ", K.eval(pred).shape)
        print("gt: ", K.eval(gt).shape)
        
        predval = tf.constant(pred.shape[1])
        gtval = tf.constant(gt.shape[1])
        print("tf.shape(pred)[1]: ", K.eval(predval))
        print("tf.shape(gt)[1]: ", K.eval(gtval))


        val = tf.shape(pred)[1] - tf.shape(gt)[1]
        print("val: ", K.eval(val))
        gt = tf.slice(gt, [0, 0, 0], [1, tf.shape(pred)[1], 18],name="rem_slice")
        return gt

    def gt_add(pred, gt):
        print("gt_add")
        print("pred: ", K.eval(pred).shape)
        print("gt: ", K.eval(gt).shape)
        
        val = tf.shape(pred)[1] - tf.shape(gt)[1]
        print("val: ", K.eval(val))
        ext = tf.slice(gt, [0, 0, 0], [1, 1, 18], name="add_slice")
        
        multiply = tf.constant([1,val,1])
        ext = tf.tile(ext, multiply)
        gt = K.concatenate([ext,gt], axis=1)
        print("pred: ", K.eval(pred).shape)
        print("gt: ", K.eval(gt).shape)
        return gt

    def equalalready(gt, pred): return pred

    def make_equal(pred, gt):
        print("tf.shape(pred)[1]: ", K.eval(tf.shape(pred)[1]))
        print("tf.shape(gt)[1]: ", K.eval(tf.shape(gt)[1]))


        equal_tensor = tf.cond(tf.shape(pred)[1] < tf.shape(gt)[1], lambda: gt_rem(pred, gt), lambda: gt_add(pred, gt), name="make_equal_cond")
        return equal_tensor


    def matcher(y_true_1,y_pred_1,y_true_2,y_pred_2, bsz):
        pred = 0
        gt = 0

        for i in range(bsz):
            
            filterer = tf.where(tf.not_equal(y_true_1[i,:,-4],99))
            filterer_2 = tf.where(tf.not_equal(y_true_2[i,:,-4],99))

            y_true_new = tf.gather_nd(y_true_1[i,:,:],filterer)            
            y_true_new = tf.expand_dims(y_true_new, 0)
            
            y_true_2_new = tf.gather_nd(y_true_2[i,:,:],filterer_2)
            y_true_2_new = tf.expand_dims(y_true_2_new, 0)

            set1 = tf.cast(y_true_new[i,:,-4],dtype=tf.int32)
            set2 = tf.cast(y_true_2_new[i,:,-4],dtype=tf.int32)
            
            id_pick = tf.sets.set_intersection(set1[None,:], set2[None, :])
            id_pick = tf.cast(id_pick.values[0],dtype=tf.float64)
                        
            filterer = tf.where(tf.equal(y_true_1[i,:,-4],id_pick))
            filterer_2 = tf.where(tf.equal(y_true_2[i,:,-4],id_pick))

            y_true_new = tf.gather_nd(y_true_1[i,:,:],filterer)            
            y_true_new = tf.expand_dims(y_true_new, 0)
            
            y_true_2_new = tf.gather_nd(y_true_2[i,:,:],filterer_2)
            y_true_2_new = tf.expand_dims(y_true_2_new, 0)
            
            print("y_pred_1:", y_pred_1[i,:,-16:-12].shape)
            print("y_true_new:", K.eval(y_true_new[i,:,-16:-12]).shape)
            print("--------------------------------------------------")

            iou_out = tf.py_func(iou, [y_pred_1[i,:,-16:-12],tf.convert_to_tensor(y_true_new[i,:,-16:-12])], tf.float64, name="iou_out")
            bipartite_matches = tf.py_func(match_bipartite_greedy, [iou_out], tf.int64, name="bipartite_matches")
            out = tf.gather(y_pred_2[i,:,:], [bipartite_matches], axis=0, name="out")
            
            print("iou_out:", K.eval(iou_out).shape)
            print("bipartite_matches:", K.eval(bipartite_matches).shape)
            print("out:", K.eval(out).shape)
            print("--------------------------------------------------")

            print("tf.shape(out)[1]:", K.eval(tf.shape(out)[1]))
            print("tf.shape(y_true_2_new)[1]:", K.eval(tf.shape(y_true_2_new)[1]))

            box_comparer = tf.reduce_all(tf.equal(tf.shape(out)[1], tf.shape(y_true_2_new)[1]), name="box_comparer")
            print("box_comparer:", K.eval(box_comparer))
            print("--------------------------------------------------")


            y_true_2_equal = tf.cond(box_comparer, lambda: equalalready(out, y_true_2_new), lambda: make_equal(out, y_true_2_new), name="y_true_cond")

            if i != 0:
                pred = K.concatenate([pred,out], axis=-1)
                gt = K.concatenate([gt,y_true_2_equal], axis=0)
            else:
                pred = out
                gt = y_true_2_equal    
        return pred, gt

        
    y_true_1 = y_true[:,:,:18]
    y_pred_1 = y_pred[:,:,:18]
    y_true_2 = y_true[:,:,18:]
    y_pred_2 = y_pred[:,:,18:]

    y_pred, y_true = matcher(y_true_1,y_pred_1,y_true_2,y_pred_2,1)
                
    y_pred = tf.cast(y_pred, dtype=tf.float32)
    y_true = tf.cast(y_true, dtype=tf.float32)

    y_pred1 = y_pred
    t_true1 = y_true

    batch_size = tf.shape(y_pred1)[0]
    n_boxes = tf.shape(t_true1)[1] 

    classification_loss = tf.to_float(log_loss(t_true1[:,:,:-16], y_pred1[:,:,:-16])) # Output shape: (batch_size, n_boxes)
    localization_loss = tf.to_float(smooth_L1_loss(t_true1[:,:,-16:-12], y_pred1[:,:,-16:-12])) # Output shape: (batch_size, n_boxes)

    negatives = t_true1[:,:,0] # Tensor of shape (batch_size, n_boxes)
    positives = tf.to_float(tf.reduce_max(t_true1[:,:,1:-16], axis=-1)) # Tensor of shape (batch_size, n_boxes)
    n_positive = tf.reduce_sum(positives)

    pos_class_loss = tf.reduce_sum(classification_loss * positives, axis=-1) # Tensor of shape (batch_size,)


    neg_class_loss_all = classification_loss * negatives # Tensor of shape (batch_size, n_boxes)
    n_neg_losses = tf.count_nonzero(neg_class_loss_all, dtype=tf.int32) # The number of non-zero loss entries in `neg_class_loss_all`
    n_negative_keep = tf.minimum(tf.maximum(neg_pos_ratio * tf.to_int32(n_positive), n_neg_min), n_neg_losses)

    def f1():
        return tf.zeros([batch_size])
    def f2():

        neg_class_loss_all_1D = tf.reshape(neg_class_loss_all, [-1]) # Tensor of shape (batch_size * n_boxes,)
        values, indices = tf.nn.top_k(neg_class_loss_all_1D,
                                      k=n_negative_keep,
                                      sorted=False) # We don't need them sorted.

        negatives_keep = tf.scatter_nd(indices=tf.expand_dims(indices, axis=1),
                                       updates=tf.ones_like(indices, dtype=tf.int32),
                                       shape=tf.shape(neg_class_loss_all_1D)) # Tensor of shape (batch_size * n_boxes,)
        negatives_keep = tf.to_float(tf.reshape(negatives_keep, [batch_size, n_boxes])) # Tensor of shape (batch_size, n_boxes)
        # ...and use it to keep only those boxes and mask all other classification losses
        neg_class_loss = tf.reduce_sum(classification_loss * negatives_keep, axis=-1) # Tensor of shape (batch_size,)
        return neg_class_loss

    neg_class_loss = tf.cond(tf.equal(n_neg_losses, tf.constant(0)), f1, f2)

    class_loss = pos_class_loss + neg_class_loss # Tensor of shape (batch_size,)

    loc_loss = tf.reduce_sum(localization_loss * positives, axis=-1) # Tensor of shape (batch_size,)

    # 4: Compute the total loss.

    total_loss = (class_loss + alpha * loc_loss) / tf.maximum(1.0, n_positive) # In case `n_positive == 0`
    total_loss = total_loss * tf.to_float(batch_size)
    total_loss.set_shape((None,))
    return total_loss


In [9]:
class prediction_history(Callback):
    def __init__(self):
        print("Predictor")
    def on_epoch_end(self, epoch, logs={}):
        predder = np.load('outputs/predder.npy')
        bX = predder[0][0]
        bZ = predder[0][1]
        gX = predder[0][2]
        gZ = predder[0][3]
        
        y_true = tf.convert_to_tensor(predder[1]['predictions_1_to_2'], dtype=tf.float64)

        intermediate_layer_model = Model(inputs=model.input,
                             outputs=model.get_layer("predictions_1").output)
        intermediate_layer_model_1 = Model(inputs=model.input,
                             outputs=model.get_layer("predictions_1_to_2").output)
        intermediate_layer_model_2 = Model(inputs=model.input,
                             outputs=model.get_layer("predictions_2").output)
        intermediate_layer_model_3 = Model(inputs=model.input,
                             outputs=model.get_layer("predictions_2_to_1").output)

        intermediate_output = intermediate_layer_model.predict([bX,bZ,gX,gZ])
        intermediate_output_1 = intermediate_layer_model_1.predict([bX,bZ,gX,gZ])
        intermediate_output_2 = intermediate_layer_model_2.predict([bX,bZ,gX,gZ])
        intermediate_output_3 = intermediate_layer_model_3.predict([bX,bZ,gX,gZ])
        
        np.save('outputs/predictions_1_e'+str(epoch)+'.npy', intermediate_output)
        np.save('outputs/predictions_1_to_2_e'+str(epoch)+'.npy', intermediate_output_1)
        np.save('outputs/predictions_2_e'+str(epoch)+'.npy',intermediate_output_2)
        np.save('outputs/predictions_2_to_1_e'+str(epoch)+'.npy',intermediate_output_3)



        y_pred = tf.cast(intermediate_output_1, dtype=tf.float64)
        loss = compute_loss(y_true, y_pred)
        
        print(K.eval(loss))
#         np.save('outputs/y_pred'+str(epoch)+'.npy',K.eval(y_pred1))
#         np.save('outputs/y_true'+str(epoch)+'.npy',K.eval(y_true1))


In [10]:
batch_size = 4

ssd_data_augmentation = SSDDataAugmentation_modified(img_height=img_height,
                                            img_width=img_width,
                                            background=mean_color)
# For the validation generator:
convert_to_3_channels = ConvertTo3Channels_Modified()  
resize = Resize_Modified(height=img_height, width=img_width)

predictor_sizes = [model.get_layer('conv4_3_norm_mbox_conf__1').output_shape[1:3],
                   model.get_layer('fc7_mbox_conf__1').output_shape[1:3],
                   model.get_layer('conv6_2_mbox_conf__1').output_shape[1:3],
                   model.get_layer('conv7_2_mbox_conf__1').output_shape[1:3],
                   model.get_layer('conv8_2_mbox_conf__1').output_shape[1:3],
                   model.get_layer('conv9_2_mbox_conf__1').output_shape[1:3]]

ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.5,
                                    normalize_coords=normalize_coords)


train_generator = train_dataset.generate(batch_size=batch_size,
                                         shuffle=False,
                                         transformations=[ssd_data_augmentation],
                                         label_encoder=ssd_input_encoder,
                                         returns={'processed_images',
                                                  'encoded_labels'},
                                         keep_images_without_gt=False)

val_generator = val_dataset.generate(batch_size=batch_size,
                                     shuffle=False,
                                     transformations=[convert_to_3_channels,
                                                      resize],
                                     label_encoder=ssd_input_encoder,
                                     returns={'processed_images',
                                              'encoded_labels'},
                                     keep_images_without_gt=False)

train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))

Number of images in the training dataset:	     4
Number of images in the validation dataset:	     2


In [11]:
model_checkpoint = ModelCheckpoint(filepath='checkpoints/exp_10_epoch-{epoch:02d}_loss-{loss:.4f}_val_loss-{val_loss:.4f}.h5',
                                   monitor='val_loss',
                                   verbose=1,
                                   save_best_only=True,
                                   save_weights_only=False,
                                   mode='auto',
                                   period=1)
#model_checkpoint.best = 
tbCallBack = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

csv_logger = CSVLogger(filename='ssd300_pascal_07+12_training_log.csv',
                       separator=',',
                       append=True)

learning_rate_scheduler = LearningRateScheduler(schedule=lr_schedule)

early_stopping = EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=1,
                              verbose=0, mode='auto')

terminate_on_nan = TerminateOnNaN()
printer_callback = prediction_history()
# custom_los = custom_loss()
callbacks = [
#             model_checkpoint,
#             csv_logger,
#             custom_los,
            learning_rate_scheduler,
            early_stopping,
            terminate_on_nan,
            printer_callback,
            tbCallBack
            ]

# If you're resuming a previous training, set `initial_epoch` and `final_epoch` accordingly.
initial_epoch   = 0
final_epoch     = 500
steps_per_epoch = 1000

history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=ceil(train_dataset_size/batch_size),
                              epochs=final_epoch,
                              callbacks=callbacks,
                              verbose=1,
                              validation_data=val_generator,
                              validation_steps=ceil(val_dataset_size/batch_size),
                              initial_epoch=initial_epoch)

Predictor
Epoch 1/500
1/1 [==============================] - 18s 18s/step - loss: 693.9352 - predictions_1_loss: 377.1191 - predictions_2_loss: 311.1080 - predictions_1_to_2_loss: -6.7851 - predictions_2_to_1_loss: -4.2506 - predictions_1_Accuracy: 0.1001 - predictions_2_Accuracy: 0.0752 - val_loss: 6878.9072 - val_predictions_1_loss: 1903.1128 - val_predictions_2_loss: 4916.6040 - val_predictions_1_to_2_loss: 16.6170 - val_predictions_2_to_1_loss: 25.9600 - val_predictions_1_Accuracy: 0.0984 - val_predictions_2_Accuracy: 0.0715
y_pred_1: (17292, 4)
y_true_new: (8, 4)
--------------------------------------------------
iou_out: (17292, 8)
bipartite_matches: (17292,)
out: (1, 17292, 18)
--------------------------------------------------
tf.shape(out)[1]: 17292
tf.shape(y_true_2_new)[1]: 13
box_comparer: False
--------------------------------------------------


ValueError: Operation 'y_true_cond/strided_slice' has been marked as not fetchable.